# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [11]:
import os
import ollama
from dotenv import load_dotenv
from scraper import fetch_website_contents, fetch_website_links
from openai import OpenAI
from IPython.display import display, Markdown, update_display


In [35]:
# constants

MODEL_GPT_4O_MINI = 'gpt-4o-mini'
MODEL_GPT_5_NANO = 'gpt-5-nano'
CLIENT_OLLAMA = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [ ]:
# set up environment

load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")
if api_key and api_key.startswith('sk-'):
    print('OPENAI_API_KEY is set')
    client = OpenAI(api_key=api_key)
else:
    raise ValueError('OPENAI_API_KEY is not set')

system_prompt = ''' 
You are a helpful assistant that can answer questions about the code.
By default, make sure to explain the answer in a way that is easy to understand.
If the user tells you that they are an advanced user or a professional, 
make sure to explain the answer in a way that is technical and detailed.
If the user tells you they are coming from a different programming language, or mentioned another programming language,
make sure to explain the answer in a way that is relevant to the mentioned language.
'''


In [ ]:
def get_local_models():
    return [model['model'] for model in ollama.list()['models']]

get_local_models()


In [ ]:
def get_models_list():
    remote_models = [{'name': MODEL_GPT_4O_MINI, 'type': 'remote'}, {'name': MODEL_GPT_5_NANO, 'type': 'remote'}]
    local_models = [{'name': model, 'type': 'local'} for model in get_local_models()]
    models_dict = {f'{i+1}': model for i, model in enumerate(remote_models + local_models)}
    return models_dict

def print_models_list():
    models_dict = get_models_list()
    return '\n'.join([f'{i}. {v.get('name')}' for i, v in models_dict.items()])

print_models_list()

In [ ]:
# here is a sample question. will be used if the user provides an empty question.

sample_question = ''' 
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
'''

In [46]:
# Get a remote model to answer, with streaming. By default, use gpt-4o-mini.
def ask_gpt(question, model=MODEL_GPT_4O_MINI):
    stream = client.chat.completions.create( 
        model=model, 
        messages=[ 
            {'role': 'system', 'content': system_prompt}, 
            {'role': 'user', 'content': question}, 
            ], 
            stream=True,
    ) 
    response = '' 
    display_handle = display(Markdown(''), display_id=True) 
    for chunk in stream: 
        response += chunk.choices[0].delta.content or '' 
        display_handle.update(Markdown(response))

In [ ]:
# Get a local model to answer. By default, use the first local model.
def ask_ollama(question, model=get_local_models()[0]): 
    user_prompt = question
    stream = CLIENT_OLLAMA.chat.completions.create(
        model=model,
        messages=[
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': user_prompt},
        ],
        stream=True,
    )
    response = ''
    display_handle = display(Markdown(''), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        display_handle.update(Markdown(response))

In [ ]:
models_list = get_models_list()
print(f'Enter the number of the model you want to use:\n{print_models_list()}')

choice = input('Enter the number of the model you want to use: ')
model = models_list.get(choice)

if not model:
    print('Invalid choice. Please try again.')
    raise ValueError('Invalid choice. Please try again.')

question = input('Enter the question you want to ask: ')

if not question:
    question = sample_question

if model['type'] == 'remote':
    print(f'\n{20 * '='}\ngenerating response using {model['name']}')
    ask_gpt(question)
else:
    print(f'\n{20 * '='}\ngenerating response using {model['name']}')
    ask_ollama(question, model=model['name'])
